<img align="right" width="200" height="200" src="https://static.tildacdn.com/tild6236-6337-4339-b337-313363643735/new_logo.png">

# Настройка, мониторинг и оптимизация Spark
**Андрей Титов**  
tenke.iu8@gmail.com  

## На этом занятии
+ Настройка spark-submit окружения
+ Работа со Spark UI
+ "Вредные" советы

## Настройка spark-submit окружения
Дистрибутив Spark содержит в себе различные библиотеки, примеры конфигурационных файлов и набор утилит. Любое Spark приложение запускается одной из утилит:
- spark-shell - запуска интерактивного `Scala REPL` с поддержкой Spark
- pyspark - запуск интеракивного `python` шела с поддержкой Spark
- spark-submit - запуск Spark приложений, собранных в виде `jar` или `py` файла с зависимостями

### Структура дистрибутива Spark
Скачаем последнюю версию с официального сайта:
https://spark.apache.org/downloads.html

In [1]:
spark

org.apache.spark.sql.SparkSession@4df68113

In [ ]:
import sys.process._

println("wget https://apache-mirror.rbc.ru/pub/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz -P lib/".!!)

Распакуем `tar.gz` архив:

In [ ]:
println("tar xvf lib/spark-2.4.5-bin-hadoop2.7.tgz -C lib/".!!)

Архив теперь можно удалить:

In [ ]:
println("rm -f lib/spark-2.4.5-bin-hadoop2.7.tgz".!!)

Изучим содержимое распакованного дистрибутива:

In [97]:
println("ls -alh lib/spark-2.4.5-bin-hadoop2.7".!!)

total 152
drwxr-xr-x   17 t3nq  staff   544B Oct 13 18:04 .
drwxr-xr-x    3 t3nq  staff    96B Oct 13 18:04 ..
-rw-r--r--    1 t3nq  staff    21K Feb  2  2020 LICENSE
-rw-r--r--    1 t3nq  staff    42K Feb  2  2020 NOTICE
drwxr-xr-x    3 t3nq  staff    96B Feb  2  2020 R
-rw-r--r--    1 t3nq  staff   3.7K Feb  2  2020 README.md
-rw-r--r--    1 t3nq  staff   187B Feb  2  2020 RELEASE
drwxr-xr-x   29 t3nq  staff   928B Oct 13 18:04 bin
drwxr-xr-x    9 t3nq  staff   288B Oct 13 19:02 conf
drwxr-xr-x    5 t3nq  staff   160B Feb  2  2020 data
drwxr-xr-x    4 t3nq  staff   128B Feb  2  2020 examples
drwxr-xr-x  228 t3nq  staff   7.1K Oct 13 18:04 jars
drwxr-xr-x    4 t3nq  staff   128B Feb  2  2020 kubernetes
drwxr-xr-x   49 t3nq  staff   1.5K Oct 13 18:04 licenses
drwxr-xr-x   19 t3nq  staff   608B Oct 13 18:04 python
drwxr-xr-x   24 t3nq  staff   768B Oct 13 18:04 sbin
drwxr-xr-x    3 t3nq  staff    96B Oct 13 18:04 yarn



`spark-submit`, `pyspark`, `spark-shell` находятся в каталоге `bin`. Когда вы запускаете свои Spark задачи, под капотом используются именно эти утилиты. Достаточно, чтобы они были установлены только на те хосты, с которых происходит запуск приложений. На обычных узлах кластера они не нужны. 

In [98]:
println("ls -alh lib/spark-2.4.5-bin-hadoop2.7/bin".!!)

total 224
drwxr-xr-x  29 t3nq  staff   928B Oct 13 18:04 .
drwxr-xr-x  17 t3nq  staff   544B Oct 13 18:04 ..
-rwxr-xr-x   1 t3nq  staff   1.1K Feb  2  2020 beeline
-rw-r--r--   1 t3nq  staff   1.0K Feb  2  2020 beeline.cmd
-rwxr-xr-x   1 t3nq  staff   5.3K Feb  2  2020 docker-image-tool.sh
-rwxr-xr-x   1 t3nq  staff   1.9K Feb  2  2020 find-spark-home
-rw-r--r--   1 t3nq  staff   2.6K Feb  2  2020 find-spark-home.cmd
-rw-r--r--   1 t3nq  staff   1.8K Feb  2  2020 load-spark-env.cmd
-rw-r--r--   1 t3nq  staff   2.0K Feb  2  2020 load-spark-env.sh
-rwxr-xr-x   1 t3nq  staff   2.9K Feb  2  2020 pyspark
-rw-r--r--   1 t3nq  staff   1.1K Feb  2  2020 pyspark.cmd
-rw-r--r--   1 t3nq  staff   1.5K Feb  2  2020 pyspark2.cmd
-rwxr-xr-x   1 t3nq  staff   1.0K Feb  2  2020 run-example
-rw-r--r--   1 t3nq  staff   1.2K Feb  2  2020 run-example.cmd
-rwxr-xr-x   1 t3nq  staff   3.1K Feb  2  2020 spark-class
-rw-r--r--   1 t3nq  staff   1.2K Feb  2  2020 spark-class.cmd
-rw-r--r--   1 t3nq  staff   2

В каталоге `python/lib` находятся зависимости для `python` - когда вы натраиваете у себя среду разработки, не нужно ставить pyspark в свой venv. Правильнее добавить эти библиотеки из дистрибутива. Так вы избежите возможных конфликтов версий в будущем

In [99]:
println("ls -alh lib/spark-2.4.5-bin-hadoop2.7/python/lib".!!)

total 1256
drwxr-xr-x   5 t3nq  staff   160B Oct 13 18:04 .
drwxr-xr-x  19 t3nq  staff   608B Oct 13 18:04 ..
-rw-r--r--   1 t3nq  staff   1.4K Feb  2  2020 PY4J_LICENSE.txt
-rw-r--r--   1 t3nq  staff    41K Feb  2  2020 py4j-0.10.7-src.zip
-rw-r--r--   1 t3nq  staff   578K Feb  2  2020 pyspark.zip



In [100]:
println("ls -alh lib/spark-2.4.5-bin-hadoop2.7/jars".!!)

total 471288
drwxr-xr-x  228 t3nq  staff   7.1K Oct 13 18:04 .
drwxr-xr-x   17 t3nq  staff   544B Oct 13 18:04 ..
-rw-r--r--    1 t3nq  staff    17K Feb  2  2020 JavaEWAH-0.3.2.jar
-rw-r--r--    1 t3nq  staff   318K Feb  2  2020 RoaringBitmap-0.7.45.jar
-rw-r--r--    1 t3nq  staff   231K Feb  2  2020 ST4-4.0.4.jar
-rw-r--r--    1 t3nq  staff    68K Feb  2  2020 activation-1.1.1.jar
-rw-r--r--    1 t3nq  staff   131K Feb  2  2020 aircompressor-0.10.jar
-rw-r--r--    1 t3nq  staff   435K Feb  2  2020 antlr-2.7.7.jar
-rw-r--r--    1 t3nq  staff   161K Feb  2  2020 antlr-runtime-3.4.jar
-rw-r--r--    1 t3nq  staff   327K Feb  2  2020 antlr4-runtime-4.7.jar
-rw-r--r--    1 t3nq  staff   4.4K Feb  2  2020 aopalliance-1.0.jar
-rw-r--r--    1 t3nq  staff    14K Feb  2  2020 aopalliance-repackaged-2.4.0-b34.jar
-rw-r--r--    1 t3nq  staff   438K Feb  2  2020 apache-log4j-extras-1.2.17.jar
-rw-r--r--    1 t3nq  staff    44K Feb  2  2020 apacheds-i18n-2.0.0-M15.jar
-rw-r--r--    1 t3nq  staff   6

В данных `jar` находятся библиотеки, включая классы драйвера и воркеров:
- org.apache.spark.repl.Main
- org.apache.spark.deploy.SparkSubmit
- org.apache.spark.executor.CoarseGrainedExecutorBackend

```scala
object SparkSubmit extends CommandLineUtils with Logging {
  override def main(args: Array[String]): Unit
}
```

```scala
private[spark] object CoarseGrainedExecutorBackend extends Logging {
    def main(args: Array[String])
}
```

```scala
object Main extends Logging {
    def main(args: Array[String]): Unit
}
```

Также в дистрибутив входят библиотеки для работы с Hadoop, встроенными источниками данных, например Parquet, и т. д.

В каталоге `conf` находятся шаблоны конфигурационных файлов:

In [101]:
println("ls -alh lib/spark-2.4.5-bin-hadoop2.7/conf".!!)

total 72
drwxr-xr-x   9 t3nq  staff   288B Oct 13 19:02 .
drwxr-xr-x  17 t3nq  staff   544B Oct 13 18:04 ..
-rw-r--r--   1 t3nq  staff   996B Feb  2  2020 docker.properties.template
-rw-r--r--   1 t3nq  staff   1.1K Feb  2  2020 fairscheduler.xml.template
-rw-r--r--   1 t3nq  staff   2.0K Feb  2  2020 log4j.properties.template
-rw-r--r--   1 t3nq  staff   7.6K Feb  2  2020 metrics.properties.template
-rw-r--r--   1 t3nq  staff   865B Feb  2  2020 slaves.template
-rw-r--r--   1 t3nq  staff   1.3K Feb  2  2020 spark-defaults.conf.template
-rwxr-xr-x   1 t3nq  staff   4.1K Feb  2  2020 spark-env.sh.template



In [106]:
println("cat lib/spark-2.4.5-bin-hadoop2.7/conf/log4j.properties.template".!!)

#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Set everything to be logged to the console
log4j.rootCategory=INFO, console
log4j.appender.console=org.apache.log4j.ConsoleAppender
log4j.appender.console.target=System.err
log4j.appender.console.layout=org.apache

**spark-defaults.conf**  
Основной конфигурационный файл Spark. В нем указываются опции запуска, зависимости, количество воркеров и т. п.

**spark-env.sh**  
Скрипт, в котором устанавливаются переменные окружения (например, `HADOOP_CONF_DIR` или `YARN_CONF_DIR`)

**log4j.properties**
Конфигурация логгеров - здесь можно задать необходимый уровень логирования для различных компонентов Spark

Изучим `spark-submit --help`:

In [ ]:
println("lib/spark-2.4.5-bin-hadoop2.7/bin/spark-submit --help".!!)

Приоритет опций (по уменьшению приоритета):
- параметры `spark-submit`
- переменные окружения
- конфигурационный файл `spark-defaults.conf`

### Выводы:
- Наличие Hadoop не является необходимым условием для работы Spark
- В Spark есть большое количество параметров, определяющих режим его работы. Большинство описано здесь: https://spark.apache.org/docs/latest/configuration.html
- Утилита `spark-submit` позволяет запустить Spark приложение как локально, так и на кластере

## Работа со Spark UI

Каждое Spark приложение по умолчанию поднимает UI, который позволяет изучить состояние задачи и провести диагностику производительности. Данные, представленные в UI, также можно получить через REST API

Получить Spark UI URL можно следующим образом:

In [ ]:
val sparkUiUrl: Option[String] = spark.sparkContext.uiWebUrl

In [ ]:
import sys.process._
sparkUiUrl.foreach( x => println(s"curl -s $x/api/v1/applications".!!))

In [ ]:
val appId = spark.sparkContext.applicationId

In [ ]:
sparkUiUrl.foreach( x => println(s"curl -s $x/api/v1/applications/$appId".!!))

Список доступных методов доступен по ссылке: https://spark.apache.org/docs/latest/monitoring.html#rest-api

Прочитаем датасет [Airport Codes](https://datahub.io/core/airport-codes):

## Вредные советы
В данной секции приведены частые ошибки, которые допускаются при работе с данными в DataFrame API.

In [2]:
// Эта нормальная :)
val csvOptions = Map("header" -> "true", "inferSchema" -> "true")
val airports = spark.read.options(csvOptions).csv("/tmp/datasets/airport-codes.csv")
airports.printSchema
airports.show(numRows = 1, truncate = 100, vertical = true)

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

-RECORD 0------------------------------------------
 ident        | 00A                                
 type         | heliport                           
 name         | Total Rf Heliport                  
 elevation_ft | 11                                 
 continent    | NA                                 
 iso_country  | US                                 
 iso_region   | US-PA                              
 municipality | Bensalem                           
 gps_code     | 00A                 

csvOptions = Map(header -> true, inferSchema -> true)
airports = [ident: string, type: string ... 10 more fields]


[ident: string, type: string ... 10 more fields]

###  Window funcs

In [12]:
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._

// val wnd = Window.partitionBy()

// val ranked = airports.select('ident, 'iso_country, 'elevation_ft, 'type, count(lit(1)).over(wnd))
// ranked.show(20, false)
// ranked.explain

val cnt = airports.count
val ranked = airports.select('ident, 'iso_country, 'elevation_ft, 'type, lit(cnt).alias("cnt"))
ranked.explain

== Physical Plan ==
*(1) Project [ident#10, iso_country#15, elevation_ft#13, type#11, 56226 AS cnt#267L]
+- *(1) FileScan csv [ident#10,type#11,elevation_ft#13,iso_country#15] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tmp/datasets/airport-codes.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<ident:string,type:string,elevation_ft:int,iso_country:string>


cnt = 56226
ranked = [ident: string, iso_country: string ... 3 more fields]


[ident: string, iso_country: string ... 3 more fields]

### Sorting

In [17]:
val ret = 
    airports
        .groupBy('iso_country)
        .agg(max('elevation_ft).alias("height"))
        .repartition(1)
        .sortWithinPartitions('height.asc)

ret.write.mode("ignore").parquet("/tmp/datasets/out")
ret.explain
println(ret.rdd.getNumPartitions)

== Physical Plan ==
*(3) Sort [height#321 ASC NULLS FIRST], false, 0
+- Exchange RoundRobinPartitioning(1)
   +- *(2) HashAggregate(keys=[iso_country#15], functions=[max(elevation_ft#13)])
      +- Exchange hashpartitioning(iso_country#15, 200)
         +- *(1) HashAggregate(keys=[iso_country#15], functions=[partial_max(elevation_ft#13)])
            +- *(1) FileScan csv [elevation_ft#13,iso_country#15] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tmp/datasets/airport-codes.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<elevation_ft:int,iso_country:string>
1


ret = [iso_country: string, height: int]


[iso_country: string, height: int]

In [13]:
spark.range(0, 10, 1, 10).repartition(1).explain

== Physical Plan ==
Exchange RoundRobinPartitioning(1)
+- *(1) Range (0, 10, step=1, splits=10)


### Dataset API

In [18]:
val jsoned = airports.toJSON
jsoned.show(10, 50)
jsoned.explain

+--------------------------------------------------+
|                                             value|
+--------------------------------------------------+
|{"ident":"00A","type":"heliport","name":"Total ...|
|{"ident":"00AA","type":"small_airport","name":"...|
|{"ident":"00AK","type":"small_airport","name":"...|
|{"ident":"00AL","type":"small_airport","name":"...|
|{"ident":"00AR","type":"closed","name":"Newport...|
|{"ident":"00AS","type":"small_airport","name":"...|
|{"ident":"00AZ","type":"small_airport","name":"...|
|{"ident":"00CA","type":"small_airport","name":"...|
|{"ident":"00CL","type":"small_airport","name":"...|
|{"ident":"00CN","type":"heliport","name":"Kitch...|
+--------------------------------------------------+
only showing top 10 rows

== Physical Plan ==
*(2) SerializeFromObject [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, input[0, java.lang.String, true], true, false) AS value#335]
+- MapPartitions <function1>, obj#334: j

jsoned = [value: string]


[value: string]

In [35]:
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Column

def toJSON[T](ds: Dataset[T]): Dataset[String] = {
    val foo: Column = to_json(struct(ds.columns.map(x => col(x)):_*))
    ds.select(foo).as[String]
}

// def
// struct(cols: Column*): Column
//  Permalink
// Creates a new struct column.

toJSON: [T](ds: org.apache.spark.sql.Dataset[T])org.apache.spark.sql.Dataset[String]


In [36]:
toJSON(airports).explain

== Physical Plan ==
Project [structstojson(named_struct(ident, ident#10, type, type#11, name, name#12, elevation_ft, elevation_ft#13, continent, continent#14, iso_country, iso_country#15, iso_region, iso_region#16, municipality, municipality#17, gps_code, gps_code#18, iata_code, iata_code#19, local_code, local_code#20, coordinates, coordinates#21), Some(Europe/Moscow)) AS structstojson(named_struct(NamePlaceholder(), ident, NamePlaceholder(), type, NamePlaceholder(), name, NamePlaceholder(), elevation_ft, NamePlaceholder(), continent, NamePlaceholder(), iso_country, NamePlaceholder(), iso_region, NamePlaceholder(), municipality, NamePlaceholder(), gps_code, NamePlaceholder(), iata_code, NamePlaceholder(), local_code, NamePlaceholder(), coordinates))#365]
+- *(1) FileScan csv [ident#10,type#11,name#12,elevation_ft#13,continent#14,iso_country#15,iso_region#16,municipality#17,gps_code#18,iata_code#19,local_code#20,coordinates#21] Batched: false, Format: CSV, Location: InMemoryFileIndex[fi

In [32]:
val df = spark.range(10)
df.select(to_json(struct(df.columns.map(x => col(x)):_*))).show

+--------------------------------------------------+
|structstojson(named_struct(NamePlaceholder(), id))|
+--------------------------------------------------+
|                                          {"id":0}|
|                                          {"id":1}|
|                                          {"id":2}|
|                                          {"id":3}|
|                                          {"id":4}|
|                                          {"id":5}|
|                                          {"id":6}|
|                                          {"id":7}|
|                                          {"id":8}|
|                                          {"id":9}|
+--------------------------------------------------+



df = [id: bigint]


[id: bigint]

In [37]:
case class Apple(size: Int, color: String)

defined class Apple


In [40]:
List(Apple(1, "red")).toDS.select('size).explain

== Physical Plan ==
LocalTableScan [size#395]


### UDF

In [50]:
val mega_udf = udf { (left: java.lang.Integer, right: String) => "ok" }

mega_udf = UserDefinedFunction(<function2>,StringType,Some(List(IntegerType, StringType)))


UserDefinedFunction(<function2>,StringType,Some(List(IntegerType, StringType)))

In [51]:
spark
    .range(1)
    .select(
        lit(1).alias("left"), 
        lit("foo").alias("right")
    )
    .select(mega_udf('left, 'right))
    .show

+----------------+
|UDF(left, right)|
+----------------+
|              ok|
+----------------+



In [52]:
spark
    .range(1)
    .select(
        lit(null).alias("left"), 
        lit("foo").alias("right")
    )
    .select(mega_udf('left, 'right))
    .show

+----------------+
|UDF(left, right)|
+----------------+
|              ok|
+----------------+



In [53]:
spark
    .range(1)
    .select(
        lit(1).alias("left"), 
        lit(null).alias("right")
    )
    .select(mega_udf('left, 'right))
    .show

+----------------+
|UDF(left, right)|
+----------------+
|              ok|
+----------------+



In [54]:
val foo: java.lang.Integer = 3
foo.toInt

foo = 3


3

In [55]:
case class Foo(first: Int, second: Int, third: Int)

defined class Foo


In [62]:
val mega_udf2 = udf { () => Thread.sleep(1000); Foo(1,2,3) }.asNondeterministic

mega_udf2 = UserDefinedFunction(<function0>,StructType(StructField(first,IntegerType,false), StructField(second,IntegerType,false), StructField(third,IntegerType,false)),Some(List()))


UserDefinedFunction(<function0>,StructType(StructField(first,IntegerType,false), StructField(second,IntegerType,false), StructField(third,IntegerType,false)),Some(List()))

In [65]:
spark.time { 
val df = spark
    .range(0, 10, 1, 1)
    .select(mega_udf2().alias("res"))
df.cache
df.count
df.select('res("first"), 'res("second"), 'res("third")).explain
df.unpersist
}

== Physical Plan ==
*(1) Project [res#576.first AS res.first#602, res#576.second AS res.second#603, res#576.third AS res.third#604]
+- InMemoryTableScan [res#576]
      +- InMemoryRelation [res#576], StorageLevel(disk, memory, deserialized, 1 replicas)
            +- *(1) Project [UDF() AS res#576]
               +- *(1) Range (0, 10, step=1, splits=1)
Time taken: 10409 ms


[res: struct<first: int, second: int ... 1 more field>]

In [64]:
spark.time { 
spark
    .range(0, 10, 1, 1)
    .select(mega_udf2().alias("res"))
    .select(col("res.*")).show
}

+-----+------+-----+
|first|second|third|
+-----+------+-----+
|    1|     2|    3|
|    1|     2|    3|
|    1|     2|    3|
|    1|     2|    3|
|    1|     2|    3|
|    1|     2|    3|
|    1|     2|    3|
|    1|     2|    3|
|    1|     2|    3|
|    1|     2|    3|
+-----+------+-----+

Time taken: 10133 ms


### Cache

In [72]:
import sys.process._
"""cp -f /tmp/datasets/source/1.txt /tmp/datasets/cache.txt/1.txt""".!
"""cp -f /tmp/datasets/source/2.txt /tmp/datasets/cache.txt/2.txt""".!

0

In [74]:
sc.uiWebUrl

Some(http://192.168.1.68:4042)

In [78]:
val df = spark.read.text("/tmp/datasets/cache.txt")
df.show(10)

df.show(1)
df.cache

+-----+
|value|
+-----+
|    1|
|    1|
|    1|
|    1|
|    1|
|    3|
|    3|
|    3|
|    3|
|    3|
+-----+

+-----+
|value|
+-----+
|    1|
+-----+
only showing top 1 row



df = [value: string]


[value: string]

In [75]:
"""cp -f /tmp/datasets/source/1.txt /tmp/datasets/cache.txt/2.txt""".!
"""cp -f /tmp/datasets/source/3.txt /tmp/datasets/cache.txt/1.txt""".!

0

In [76]:
df.show(20)

+-----+
|value|
+-----+
|    2|
|    2|
|    2|
|    2|
|    2|
|    3|
|    3|
|    3|
|    3|
|    3|
+-----+



In [ ]:
spark.read.text("/tmp/datasets/cache.txt").show

In [81]:
spark.sharedState.cacheManager.clearCache

### Coalesce

In [94]:
val mega_udf2 = udf { () => Thread.sleep(1000); Foo(1,2,3) }

mega_udf2 = UserDefinedFunction(<function0>,StructType(StructField(first,IntegerType,false), StructField(second,IntegerType,false), StructField(third,IntegerType,false)),Some(List()))


UserDefinedFunction(<function0>,StructType(StructField(first,IntegerType,false), StructField(second,IntegerType,false), StructField(third,IntegerType,false)),Some(List()))

In [96]:
spark.time {
val ret = spark.range(0, 10, 1, 4).withColumn("foo", mega_udf2())
ret.show
ret.explain
}

+---+---------+
| id|      foo|
+---+---------+
|  0|[1, 2, 3]|
|  1|[1, 2, 3]|
|  2|[1, 2, 3]|
|  3|[1, 2, 3]|
|  4|[1, 2, 3]|
|  5|[1, 2, 3]|
|  6|[1, 2, 3]|
|  7|[1, 2, 3]|
|  8|[1, 2, 3]|
|  9|[1, 2, 3]|
+---+---------+

== Physical Plan ==
*(1) Project [id#864L, UDF() AS foo#866]
+- *(1) Range (0, 10, step=1, splits=4)
Time taken: 5113 ms


In [84]:
spark.time {
val ret = spark.range(0, 10, 1, 2).withColumn("foo", mega_udf2())
ret.cache
ret.count
ret.coalesce(1).show
ret.coalesce(1).explain
}

+---+---------+
| id|      foo|
+---+---------+
|  0|[1, 2, 3]|
|  1|[1, 2, 3]|
|  2|[1, 2, 3]|
|  3|[1, 2, 3]|
|  4|[1, 2, 3]|
|  5|[1, 2, 3]|
|  6|[1, 2, 3]|
|  7|[1, 2, 3]|
|  8|[1, 2, 3]|
|  9|[1, 2, 3]|
+---+---------+

== Physical Plan ==
Coalesce 1
+- InMemoryTableScan [id#709L, foo#711]
      +- InMemoryRelation [id#709L, foo#711], StorageLevel(disk, memory, deserialized, 1 replicas)
            +- *(1) Project [id#709L, UDF() AS foo#711]
               +- *(1) Range (0, 10, step=1, splits=2)
Time taken: 5225 ms


In [88]:
spark.sharedState.cacheManager.clearCache